<a href="https://colab.research.google.com/github/Justduzi/TF_NLP/blob/main/NLPTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Nlp Fundamentals in Twnsorflow

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-11-14 12:16:05--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-11-14 12:16:05 (96.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
#Kaggle NLP with disaster tweets dataset
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-11-14 12:19:20--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.145.128, 173.194.69.128, 173.194.79.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.28MB/s    in 0.5s    

2022-11-14 12:19:21 (1.28 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



##Visualize dataset


In [50]:
import pandas as pd
train_df =pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [51]:
# shuffle training dataframe
train_df_shuffled= train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [52]:
#how many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [53]:
#how many examples in test and train df
len(train_df), len(test_df)

(7613, 3263)

In [54]:
#random examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Ignition Knock (Detonation) Sensor-Senso Standard KS57 http://t.co/bzZdeDcthL http://t.co/OQJNUyIBxM

---

Target: 0 (not real disaster)
Text:
@lucypalladino and I don't have any classes together and I'm not sure if I'll be able to survive

---

Target: 0 (not real disaster)
Text:
Cyclist who collided with runner on Roanoke greenway wins $300000 civil verdict http://t.co/WgasoeNCwc via @roanoketimes

---

Target: 0 (not real disaster)
Text:
@JYHeffect my good you stay in NY??? ?

---

Target: 0 (not real disaster)
Text:
@blanksocietyx @emmerdale IM SCREAMING HES MY FAVOURITE

---



#SPlit training dataset to create validation dataset

In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [56]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

Tokenization of text with tensorflow


In [57]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None, standardize="lower_and_strip_punctuation",
                                    split="whitespace", ngrams=None,
                                    output_mode="int", output_sequence_length=None)

In [58]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [59]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [60]:
#fit the text vectorization
text_vectorizer.adapt(train_sentences)

In [61]:
#Test tokenization
sample_sentence= "there is a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [62]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Are people not concerned that after #SLAB's obliteration in Scotland #Labour UK is ripping itself apart over #Labourleadership contest?      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  22,   57,   34, 1696,   16,   43, 8432,  536,    4, 3437,    1,
         915,    9, 9066, 2418]])>

In [63]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most cmommon words: {top_5_words}")
print(f"least most cmommon words: {bottom_5_words}")

Number of words in vocab: 10000
5 most cmommon words: ['', '[UNK]', 'the', 'a', 'in']
least most cmommon words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating an embedding layer in tensorflow

In [64]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,output_dim=128,
                            input_length=max_length)

In [65]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@YoungHeroesID LAVA BLAST dan POWER RED #PantherAttack @Mirmanda11 @evaaaSR      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00558976,  0.01243623,  0.01722619, ..., -0.0468292 ,
         -0.04593035, -0.0388102 ],
        [-0.00187409,  0.02912594,  0.04163477, ..., -0.04505261,
         -0.03926396,  0.00138617],
        [-0.0209339 , -0.01706716, -0.00089891, ...,  0.0350317 ,
          0.02231884, -0.04585114],
        ...,
        [-0.03166263, -0.04504795,  0.04009311, ...,  0.0194072 ,
          0.00387057, -0.00304307],
        [-0.03166263, -0.04504795,  0.04009311, ...,  0.0194072 ,
          0.00387057, -0.00304307],
        [-0.03166263, -0.04504795,  0.04009311, ...,  0.0194072 ,
          0.00387057, -0.00304307]]], dtype=float32)>

In [66]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))


Original text:
@ArianaGrande @justinbieber I'M SCREAMING OMG #IDOLS #22DAYS #WDYM      

Embedded version:


In [67]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 4.7828231e-02, -2.9789735e-02, -1.1334967e-02, -1.7479680e-02,
       -3.1655002e-02,  3.9509211e-02, -4.7516823e-03,  3.9208781e-02,
       -1.3531793e-02,  4.3179128e-02, -2.4107743e-02,  3.0368242e-02,
        3.5807621e-02, -2.3888839e-02,  3.3144999e-02,  2.2476498e-02,
       -4.6740472e-02, -2.9445812e-04, -3.2106150e-02,  2.9904630e-02,
       -1.2597956e-02, -3.2583587e-03, -2.8332328e-02,  4.5007434e-02,
       -3.3220828e-02,  2.8833151e-03, -4.1957926e-02, -1.8037058e-02,
        9.3050227e-03,  4.9367752e-02,  4.9763918e-04, -4.7055494e-02,
       -4.3716278e-02,  2.4082188e-02,  4.8406277e-02, -2.0828748e-02,
        2.7905252e-02,  1.3577830e-02, -1.3952244e-02, -2.7722120e-03,
       -4.8275888e-02,  2.6272621e-02, -4.8307478e-02, -5.9408061e-03,
        2.5671270e-02, -1.6456746e-02, -4.3574095e-02,  4.1099414e-03,
       -4.9425911e-02, -1.9599319e-02, -3.4914505e-02, -2.0002758e-02,
        2.4161879e-02, -9.611

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data




### Model 0: getting the baseline


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#create tokenization and pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])
#fit the model
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [69]:
#evaluate baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [70]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [71]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [72]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Model 2 Feed Forward Neutral

In [73]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [81]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x= embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs= layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="Model_1")

In [84]:
model_1.summary()

Model: "Model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-tra

In [82]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [83]:
history_1 = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20221114-151737
Epoch 1/5
215/215 [==============================] - 4s 5ms/step - loss: 0.6113 - accuracy: 0.6941 - val_loss: 0.5360 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4410 - accuracy: 0.8194 - val_loss: 0.4751 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3464 - accuracy: 0.8628 - val_loss: 0.4573 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2843 - accuracy: 0.8920 - val_loss: 0.4628 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2369 - accuracy: 0.9124 - val_loss: 0.4798 - val_accuracy: 0.7861


In [85]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4798 - accuracy: 0.7861


[0.4798266887664795, 0.7860892415046692]